In [ ]:
import sys
import os
import torch

os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']='0'

import numpy as np
import plotly.graph_objects as go

In [ ]:
def toDisplay(x, target_dim = 2):
    while(x.dim() > target_dim):
        x = x[0]
    return x.detach().cpu().numpy()

In [ ]:
import sys
sys.path.insert(1, '/home/exx/Documents/equivariant_pose_graph/python')
from equivariant_pose_graph.models.transformer_flow import ResidualFlow, ResidualFlow_V1
from equivariant_pose_graph.training.flow_equivariance_training_module import EquivarianceTrainingModule
 
from ndf_robot.eval.test_trained_model_place import load_data

In [ ]:
network = ResidualFlow()

model = EquivarianceTrainingModule(
    network)
model.cuda()
# checkpoint_file = "/home/exx/media/DataDrive/singularity_chuerp/equiv_pgraph_logs/train_test_mr_dcpflow_residual0/equiv_dcpflow/version_9/checkpoints/epoch=410-step=51375.ckpt"
checkpoint_file = '/home/exx/media/DataDrive/singularity_chuerp/equiv_pgraph_logs/train_test_mr_dcpflow_residual0_attn_trans0.1_rot10_meancenter_overfit/equiv_dcpflow/version_1/saved_checkpts/epoch=24-step=3125.ckpt'
model.load_state_dict(torch.load(checkpoint_file)['state_dict'])
 
##
place_model = model

In [ ]:
# from pathlib import Path
# import torch.nn.functional as F

# num_classes = 3
# data_idx = 0
# cloud_types = ['init', 'pre_grasp', 'post_grasp', 'teleport', 'post_place', 'final']
# cloud_type = 'teleport'
# # data_path = Path('/home/bokorn/src/ndf_robot/notebooks')
# # point_data = np.load(data_path / f'{data_idx}_obj_points.npz')
# # data_path = Path('/home/exx/Documents/ndf_robot/train_new_data_3/renders')
# data_path = Path('/home/exx/Documents/ndf_robot/src/ndf_robot/debug_place')
# point_data = np.load(data_path / f'{data_idx}_{cloud_type}_obj_points.npz', allow_pickle = True)

# points_raw_np = point_data['clouds']
# classes_raw_np = point_data['classes']
 
# class_0 = 0
# class_1 = 1
# class_2 = 2
# num_classes = 2
# mask = np.isin(classes_raw_np, [class_0, class_1])
# points_np = points_raw_np[mask].copy()
# classes_np = np.where(classes_raw_np[mask] == class_0, 0, 1)
 
# print(classes_np.shape)

# # points_np = points_np - points_np[classes_np==class_1].mean(axis=0)
# points_np = points_np - points_np[classes_np==class_0].mean(axis=0)

# points = torch.from_numpy(points_np).unsqueeze(0).float().cuda()
# class_ids = torch.from_numpy(classes_np).unsqueeze(0).long().cuda()
# classes = F.one_hot(class_ids, num_classes=num_classes)
# points = points[:,::2]
# classes = classes[:,::2]
# # object_points = []
# # for j in range(num_classes):
# #     object_points.append(points[class_ids==j,:])
# # colors_np = point_data['colors']
# print(points_np.shape)


In [ ]:
def plot(points_action, points_anchor):
    colors = [
    '#1f77b4',  # muted blue
    '#ff7f0e',  # safety orange
    '#2ca02c',  # cooked asparagus green
    '#d62728',  # brick red
    '#9467bd',  # muted purple
    '#8c564b',  # chestnut brown
    '#e377c2',  # raspberry yogurt pink
    '#7f7f7f',  # middle gray
    '#bcbd22',  # curry yellow-green
    '#17becf'   # blue-teal
]
    skip = 1
    points_action_dp = toDisplay(points_action)
    points_anchor_dp = toDisplay(points_anchor)
    go_data=[
        go.Scatter3d(x=points_action_dp[::skip,0], y=points_action_dp[::skip,1], z=points_action_dp[::skip,2], 
                     mode='markers', marker=dict(size=1, color=colors[0],
                     symbol='circle')),
        go.Scatter3d(x=points_anchor_dp[::skip,0], y=points_anchor_dp[::skip,1], z=points_anchor_dp[::skip,2], 
                     mode='markers', marker=dict(size=1, color=colors[1],
                     symbol='circle')),
    ]
    layout = go.Layout(
        scene=dict(
            aspectmode='data'
        )
    )

    fig = go.Figure(data=go_data, layout=layout)
    fig.show()

In [ ]:
trans_var = 1.0
rot_var = 90

In [ ]:
from pathlib import Path
import torch.nn.functional as F
from equivariant_pose_graph.utils.se3 import random_se3

num_classes = 3
data_idx = 0
cloud_types = ['init', 'pre_grasp', 'post_grasp', 'teleport', 'post_place', 'final']
cloud_type = 'teleport'
# data_path = Path('/home/bokorn/src/ndf_robot/notebooks')
# point_data = np.load(data_path / f'{data_idx}_obj_points.npz')
data_path = Path('/home/exx/Documents/ndf_robot/train_data_3/renders')
# data_path = Path('/home/exx/Documents/ndf_robot/src/ndf_robot/debug_place')
point_data = np.load(data_path / f'{data_idx}_{cloud_type}_obj_points.npz', allow_pickle = True)

points_action, points_anchor = load_data(num_points=1024, clouds = point_data['clouds'] ,classes = point_data['classes'], action_class= 0, anchor_class= 1)
T0 = random_se3(1, rot_var = np.pi/180 * rot_var, trans_var = trans_var, device = points_action.device) 
T1 = random_se3(1, rot_var = np.pi/180 * rot_var, trans_var = trans_var,  device = points_anchor.device) 

points_action_trans = T0.transform_points(points_action)
points_anchor_trans = T1.transform_points(points_anchor)
pred_T_action, pred_T_anchor, pred_T  = place_model(points_action_trans, points_anchor_trans)
 
pred_T_action_points_action = pred_T_action.transform_points(points_action_trans)
pred_T_action_points_anchor = pred_T_anchor.transform_points(points_anchor_trans)
print(pred_T_action_points_action.shape)
# pred_T_points_action = pred_T.transform_points(points_action)
# pred_T_action_inverse_points_action = pred_T_anchor.inverse().transform_points(points_action)

In [ ]:
plot(points_action, points_anchor)

In [ ]:
plot(points_action_trans, points_anchor_trans)

In [ ]:
plot(pred_T_action_points_action, points_anchor_trans)

In [ ]:
plot(pred_T_action_points_anchor, points_action_trans)

In [ ]:
points_action, points_anchor = load_data(num_points=1024, clouds = point_data['clouds'] ,classes = point_data['classes'], action_class= 0, anchor_class= 1)
pred_T_action, pred_T_anchor, pred_T  = place_model(points_action, points_anchor)
 
pred_T_action_points_action = pred_T_action.transform_points(points_action)
for i in range(20):
    pred_T_action, pred_T_anchor, pred_T  = place_model(pred_T_action_points_action, points_anchor)
    pred_T_action_points_action = pred_T_action.transform_points(pred_T_action_points_action)
plot(pred_T_action_points_action, points_anchor)